# This is a tutorial to run a simulation of a simple Probabilistic Boolean Network


In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

# import booleanNetwork module from ./src
import sys
sys.path.append('./src')
from KGBN import booleanNetwork as bn
from KGBN import PBN as pbn
from KGBN import BMatrix

## Define a Probabilistic Boolean Network

In [22]:
# Probabilistic Boolean Network from :
#https://academic.oup.com/bioinformatics/article-pdf/18/2/261/48850623/bioinformatics_18_2_261.pdf

# number of nodes
ngenes = 3

# number of functions per node
nf = np.array( [2, 1, 2] ) 


# conectivity matrix
varF = np.array( [[0, 1, 2],  # indices of genes connected to gene 0
                  [0, 1, 2],  
                  [0, 1, 2], # indices of genes connected to gene 1
                  [0, 1, 2],
                  [0, 1, 2]] ) # indices of genes connected to gene 2

# truth tables
F = np.array( [[0, 1, 1, 1, 0, 1, 1, 1], # truth table for gene 0 
               [0, 1, 1, 0, 0, 1, 1, 1], 
               [0, 1, 1, 0, 1, 1, 0, 1], # truth table for gene 1
               [0, 0, 0, 1, 0, 1, 1, 1], # truth table for gene 2
               [0, 0, 0, 0, 0, 0, 0, 1] ]) 

# probabilities of selecting functions per node
cij = np.array([ [0.6, 0.4,-1], 
                 [1.0,-1,-1],
                 [0.5, 0.5,-1] ] )

# initial state
x0  =  np.array( [0, 1, 1] )  # initial state [v0, v1, v2] 

In [23]:
network = pbn.ProbabilisticBN( ngenes, varF, nf, F, cij, x0  ) # create a PBN object

### Alternatively, load a PBN from a file


In [21]:
x0  =  np.array( [0, 1, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_file("./input_files/examplePBN.txt", initial_state = x0)

### ... or from a string

In [31]:
network_string = """
x1 = (x1 | x2 | x3) & (!x1 | x2 | x3), 0.6
x1 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | x2 | x3), 0.4
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3), 1
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 1, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)

## Run 3 steps of simulations of PBNs

In [33]:
y = network.update( 3 )  # run 3 steps of the Probabilistic Boolean network
y

array([[1, 0, 1],
       [1, 1, 0],
       [1, 0, 1],
       [1, 1, 1]], dtype=int8)

## Run 3 steps of simulations of PBNs with noise

In [34]:
network = pbn.ProbabilisticBN( ngenes, varF, nf, F, cij, x0  ) # create a PBN object

In [37]:
noise_level = 0.01 # noise
y = network.update_noise( noise_level,  3 )  # run 3 steps of the Probabilistic Boolean network
y

array([[1, 0, 0],
       [0, 1, 0],
       [1, 1, 0],
       [1, 0, 0]], dtype=int8)

# Define mutations 
## in files/strings for PBNs

If an equation is a constant value (0 or 1), meaning that the gene is set as mutated/perturbed, the code will set the gene's value to the constant value.

In [7]:
# This was the original network
network_string = """
x1 = (x1 | x2 | x3) & (!x1 | x2 | x3), 0.6
x1 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | x2 | x3), 0.4
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3), 1
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 0, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)
network.update(3)


array([[0, 0, 1],
       [1, 1, 0],
       [1, 0, 1],
       [1, 1, 1]], dtype=int8)

In [10]:
# Now we set the first gene to 0 constant
network_string = """
x1 = 0, 1
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3), 1
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 0, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)
network.update(3)

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=int8)

In [5]:
# the ',1' can be omitted if there is a single function for that gene
network_string = """
x1 = 0
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3)
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 0, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)
network.update(3)

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=int8)

In [14]:
# the constant value can also come with a probability
network_string = """
x1 = 0, 0.5
x1 = 1, 0.5
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3), 1
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 0, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)
network.update(10)

array([[1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 1, 0]], dtype=int8)

## Mutations: update with noise

In [14]:
# this was the previous behavior
# where the noise was applied to all nodes
network_string = """
x1 = 0, 1
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3), 1
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 0, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)

# run with noise
network.update_noise(0.2, 10)

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [1, 0, 0]], dtype=int8)

In [3]:
# This is after adding a mask to update_noise
network_string = """
x1 = 0, 1
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3), 1
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 0, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)

# run with noise
network.update_noise(0.5, 10)

array([[0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1]], dtype=int8)

## The knockout function

In [2]:
# This was the original network
network_string = """
x1 = (x1 | x2 | x3) & (!x1 | x2 | x3), 0.6
x1 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | x2 | x3), 0.4
x2 = (x1 | x2 | x3) & (x1 | !x2 | !x3) & (!x1 | !x2 | x3), 1
x3 = (!x1 & x2 & x3) | (x1 & !x2 & x3) | (x1 & x2 & !x3) | (x1 & x2 & x3), 0.5
x3 = (x1 & x2 & x3), 0.5
"""
x0  =  np.array( [0, 0, 1] )  # initial state [v0, v1, v2] 
network = BMatrix.load_pbn_from_string(network_string, initial_state = x0)
network.nodeDict


{'x1': 0, 'x2': 1, 'x3': 2}

In [3]:
# simulate with noise 
network.update_noise(0.2, 10)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=int8)

In [4]:
# # Knockout the first gene
network.knockout('x1',0)
network.update_noise(0.2, 10)

array([[0, 1, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 0]], dtype=int8)

## Simulating a large network
### prob = 1 are not the same as BN

The BN and PBN implementations produce different results even with identical seeds and probability values due to structural differences in their internal representations:   

- BN uses direct 1:1 mapping between nodes and functions.
- PBN uses a more complex indexing through a cumulative sum array to handle multiple potential functions per node.   

These differences affect how random number generators advance through their sequence and how node updates propagate through the network. As iterations progress, these small implementation differences compound through the network's state transitions.


In [2]:
# Results for BN
file = 'input_files/pancreatic_vundavilli_2020_fig3.txt'
x0 = np.zeros(38, dtype=np.int8)
step = 100000
network = BMatrix.load_network_from_file(file, initial_state = x0)
np.random.seed(99)
network_traj = network.update_noise(0.05, step)
cal_range = network_traj[step//2:step]
average_array = np.mean(cal_range, axis=0)
average_array

array([0.49074, 0.49652, 0.50636, 0.49204, 0.50324, 0.50672, 0.4913 ,
       0.63718, 0.4994 , 0.5017 , 0.49868, 0.50172, 0.50878, 0.7579 ,
       0.76436, 0.7481 , 0.75808, 0.75256, 0.7574 , 0.77318, 0.76048,
       0.76068, 0.77096, 0.77406, 0.77258, 0.49926, 0.23248, 0.51414,
       0.48428, 0.4886 , 0.77936, 0.21964, 0.7764 , 0.49294, 0.76522,
       0.75   , 0.76072, 0.76448])

In [3]:
# Create a PBN version of the same network by adding probability 1 to each function
pbn_string = ""
with open(file, 'r') as f:
    for line in f:
        line = line.strip()
        if line:  # Skip empty lines
            pbn_string += line + ", 1\n"

# Initialize the PBN with the same initial state
pbn_network = BMatrix.load_pbn_from_string(pbn_string, initial_state=x0)
np.random.seed(99)
pbn_traj = pbn_network.update_noise(0.05, step)
pbn_cal_range = pbn_traj[step//2:step]
pbn_average_array = np.mean(pbn_cal_range, axis=0)
pbn_average_array

array([0.49074, 0.49652, 0.50636, 0.49204, 0.50324, 0.50672, 0.4913 ,
       0.63718, 0.4994 , 0.5017 , 0.49338, 0.50288, 0.49978, 0.76184,
       0.6493 , 0.58178, 0.5766 , 0.5608 , 0.54482, 0.77734, 0.53624,
       0.53194, 0.7241 , 0.62952, 0.62856, 0.49926, 0.4195 , 0.60764,
       0.43294, 0.46618, 0.74138, 0.26864, 0.5425 , 0.41902, 0.43786,
       0.43758, 0.44528, 0.5157 ])